In [1]:
import requests
import json
import time
import jsonlines
import os

import requests
from lxml.html import fromstring
from itertools import cycle
#import traceback

In [2]:
# we should define the size of LA
# for each query, the limitation of query will be 531MB
# therefore, we will change lat and lon 0.1 for each times

class osm:
    def __init__(self, lat_min, lon_min, lat_max, lon_max, filepath):
        self.lat_range = [lat_min, lat_max]
        self.lon_range = [lon_min, lon_max]
        self.overpass_url = "http://overpass-api.de/api/interpreter"
        self.filepath = filepath
        
    def get_poi(self):
        self.poi = []
        self.count = 0
        
        #delete the file if it exists
        if os.path.isfile(self.filepath):
            os.remove(self.filepath)
        else:
            pass
        
        #generate various ip
        proxies = self.get_proxies()
        proxy_pool = cycle(proxies)
        
        for lat in range(int(self.lat_range[0]*10000),int(self.lat_range[1]*10000),1000):
            lat = lat/10000
            lat_min, lat_max = lat, lat+0.1
            for lon in range(int(self.lon_range[0]*10000),int(self.lon_range[1]*10000),1000):
                self.writer = jsonlines.open(self.filepath, mode='a')
                self.count += 1
                lon = lon/10000
                lon_min, lon_max = lon, lon+0.1

                area = (lon_min, lat_min, lon_max, lat_max)

                # overpass api
                overpass_query = f"""
                [out:json];
                area["ISO3166-1"="DE"][admin_level=2];
                (node{area};
                 way{area};
                 rel{area};
                );
                out center;
                """
                count = 0
                while count <15:
                    try:
                        proxy = next(proxy_pool)
                        response = requests.get(self.overpass_url,
                                                params={'data': overpass_query}, 
                                                proxies={"http": proxy, "https": proxy})

                        #print(response.json()["elements"])
                        self.poi = response.json()["elements"]
                        self.writer.write_all(self.poi)
                        
                        break
                    except:
                        count +=1
                        print("connect error", proxy)
                        print(f"try_count: {count}")
                print(f"\r request_times: {self.count}", end = "")


                time.sleep(10)
                self.writer.close()
    
    def get_proxies(self):
        url = 'https://free-proxy-list.net/'
        response = requests.get(url)
        parser = fromstring(response.text)
        proxies = set()
        for i in parser.xpath('//tbody/tr')[:30]:
            if i.xpath('.//td[7][contains(text(),"yes")]'):
                #Grabbing IP and corresponding PORT
                proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
                proxies.add(proxy)
        return proxies
        

In [3]:
if __name__ == "__main__":
    model = osm(-118.6386,33.4658, -117.6100,34.3208, "poi.jsonl")
    model.get_poi()

 request_times: 1connect error 161.202.226.194:8123
try_count: 1
 request_times: 4connect error 200.69.79.218:55443
try_count: 1
connect error 51.254.237.77:3129
try_count: 2
 request_times: 5connect error 45.76.52.195:8081
try_count: 1
connect error 103.255.53.98:56031
try_count: 2
 request_times: 6connect error 161.202.226.194:8123
try_count: 1
 request_times: 7connect error 167.172.184.166:43132
try_count: 1
connect error 178.33.251.230:3129
try_count: 2
 request_times: 11connect error 103.255.53.98:56031
try_count: 1
 request_times: 12connect error 161.202.226.194:8123
try_count: 1
 request_times: 15connect error 200.69.79.218:55443
try_count: 1
 request_times: 16connect error 51.255.103.170:3129
try_count: 1
connect error 45.76.52.195:8081
try_count: 2
connect error 103.255.53.98:56031
try_count: 3
 request_times: 17connect error 161.202.226.194:8123
try_count: 1
 request_times: 18connect error 167.172.184.166:43132
try_count: 1
connect error 178.33.251.230:3129
try_count: 2
 requ

In [6]:
class google_api:
    def __init__(self, api_key):
        self.api_key = api_key
    def text_query(self, text):
        url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?\
            input={text}\
            &inputtype=textquery&fields=photos,formatted_address,name,rating,opening_hours,geometry&key={self.api_key}"
        r = requests.get(url)
        
        return r.json()
    
    def detail_text_query(self, place_id):
        url = f"https://maps.googleapis.com/maps/api/place/details/json?\
        place_id={place_id}&fields=name,rating,formatted_phone_number&key={self.api_key}"
        
        r = requests.get(url)
        return r.json()
    
    def r_geocoding(self, lat, lon):
        url =f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={self.api_key}"
        
        r = requests.get(url)
        return r.json()